# What is The Best Neighborhood In Pittsburgh?
by Casey Dinan, Kyle Bello, and Nate Quigley

Food is what brings all people together. It is the driving force of energy, happiness, and life. Without food, we are nothing.

Our project today deals with exactly that: food. Specifically, food in Alleghany County.

Coming into this project, we did had no idea what our metric was going to be. After being inspired by a dataset about Fish Fry Locations in Alleghany County, our group went down the rabit hole of food-adjacent datasets to found on the WPRDC. After finding some compelling data and interesting datasets, we figured out what best meant to us. Our metric for the best town is Pittsburgh is:

#### Food and Food Availibility

What may seem to be a rather mundane metric, food and food availibilty in Alleghany county is more interesting than it seems. Our metric involves measuring how availible food is in every town, using data from grocery stores to fast food. We came up with this metric after finding multiple datasets dealing with food availiblty by zip code/county.

### Datsets:
1. [Supermarkets/Convenience Stores in Allegeny County](https://data.wprdc.org/dataset/allegheny-county-fast-food)
2. [Fast Food Establishments in Allegheny County](https://data.wprdc.org/dataset/allegheny-county-fast-food)
3. [Farmers Markets In Allegheny County]()

These datasets allow us to look at each town in Pittsburgh, figure out how much food is availible by location, and decide the quality of said food by referencing it with obesity rates of each town. With that, lets take a look at our findings!

# [1. Allegeny County Markets](./SupermarketsAnalysis.ipynb)
By: Casey Dinan

## Results
From this dataset we can conclude that, by our metric, the towns that have the best are:

1. Shadyside
2. Squirrel Hill
3. East Liberty
4. Point Breeze

Each of these either contains the greatest number of supermarkets, or is in close proximity to markets in neighboring neighborhoods. Due to the wide availability in these neighborhoods, they came out on top in the supermarket metric. 

## Data Conclusion
This data set provided a vast amount of information on all food facilities in Alleghany County. This provided the ability to extract any relevant data from the set, in this case information regarding supermarkets. Not only did the data provide an accurate reflection of Pittsburgh, but the extensive details it provided allowed for insightful conclusions to be drawn. 

# [2. Alleghany County Fast Food Establishments](./fastFoodData.ipynb)
By: Kyle Bello

## Results

Looking at this dataset, we found these towns to have the most food options:

1. Strip District (Downtown)
2. Oakland (North Oakland, South Oakland, Shadyside)
3. Hill Disrict (Middle Hill, Upper Hill, Crawford-Roberts)
4. Pittsburgh (Squirrel Hill, Greenfield)

Look at the previous dataset, Squirrel Hill and Shadyside again make an appearance. Currently, Shadyside offers the best proximity to supermakrets and is the closest to more fast food.

## Data Conclusion
This dataset offered a lot of great information about fast food establishments in Alleghany County. While it was not the most focused to Pittsburgh, it still porvided plenty of useful information regarding fast food/restaurant availibilty in and around Pittsburgh. As well, it was fairly simple to use with a shape file of the city, as both contained zip code information. Overall, this dataset was a great addition to our project.

# [3. Alleghany Farmers Markets](./FarmersMarkerts.ipynb)
By: Nate Quigley

## Results

1. Point Breeze North
2. Point Breeze
3. Homewood West
4. Homewood South

All four of these locations are in the same vicinity of each other, and compared to the first data set above, Point Breeze seems to appear a second time, while for this specific data set, Point Breeze North claims the most Farmer's Market options.

## Data Conclusion

The dataset on farmers markets in Pittsburgh provided me with valuable information on the distribution of fresh food across different neighborhoods. Using this dataset, I was able to identify the neighborhoods with the highest concentration of farmers markets, which helped me to determine the neighborhoods with the best food availability, hence higher food quality.

## Overarching Metric

Our overall metric is food quality per neighboorhood, which is a composite ranking of the three subrankings. To properly quantify this, the intersection of all our metrics can be taken. This will allow neighboorhoods with all factors to be prioritized, as every metric is important to the overall food quality in a given area. In the event that neighboorhoods have the same amount of intersections, the rankings within the lists they occured on will be taken into account as a tie breaker.

## Results

1. Shadyside
2. Point Breeze
3. Squirrel Hill

## Data Conclusion

The overall composite rankings yeild interesting results. While it would be assumed that the more developed downtown areas of Pittsburgh would provide the greatest diversity of food options, it is actually the more suburban areas such as Shadyside and Squirrel Hill. Downtown had a plethora of fast food options, but markets in general appear to be consintrated outside the urban centers. This is a good indication of why Pittsburgh's suberbs are so diserable for both young adults and families alike, as they provide far superior food options to the rest of the Pittsburgh area.